In [50]:
import pandas as pd

In [74]:
df = pd.read_csv('data/tr_reviews_with_embeddings.csv')

In [75]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv() 

API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
RESOURCE_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

client = AzureOpenAI(
  api_key=API_KEY,
  api_version="2023-05-15",
  azure_endpoint=RESOURCE_ENDPOINT
)

### Embedding-Based Classification

In [53]:
from ast import literal_eval
import numpy as np

df["ada_v2"] = df.ada_v2.apply(literal_eval).apply(np.array)

In [54]:
EMBEDDING_MODEL = "text-embedding-ada-002"

In [71]:
import pandas as pd
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go

def generate_embeddings(text, model):
    embedding = client.embeddings.create(input=[text], model=model).data[0].embedding
    return np.array(embedding)

def generate_label_embeddings(labels, model):
    return np.array([generate_embeddings(label, model=model) for label in labels])

def compute_label_score(review_embedding, label_embeddings):
    positive_similarity = cosine_similarity([review_embedding], [label_embeddings[1]])[0][0]
    negative_similarity = cosine_similarity([review_embedding], [label_embeddings[0]])[0][0]
    return positive_similarity - negative_similarity

label_embeddings = generate_label_embeddings(['negative', 'positive'], EMBEDDING_MODEL)

def evaluate_embeddings(df):
    global label_embeddings
    df['probas'] = df['ada_v2'].apply(lambda x: compute_label_score(x, label_embeddings))
    df['preds'] = df['probas'].apply(lambda x: 'positive' if x > 0 else 'negative')

    report = classification_report(df['Sentiment'], df['preds'], output_dict=True)
    df_report = pd.DataFrame(report).transpose()

    precision, recall, _ = precision_recall_curve(df['Sentiment'], df['probas'], pos_label='positive')
    auc_score = average_precision_score(df['Sentiment'], df['probas'], pos_label='positive')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=recall, y=precision, fill='tozeroy', mode='lines', line=dict(width=2), name='Precision-Recall Curve'))
    fig.update_layout(title=f'Sentiment Classification Performance: Precision-Recall Curve (AUC: {auc_score:.2f})',
                      xaxis_title='Recall',
                      yaxis_title='Precision',
                      template='plotly_white',
                      font=dict(size=12))
    fig.show()

    return df_report.style.format("{:.2f}").background_gradient(cmap='coolwarm', subset=['precision', 'recall', 'f1-score'])

In [72]:
evaluate_embeddings(df)

,precision,recall,f1-score,support
negative,0.89,0.55,0.68,237.00
positive,0.87,0.98,0.92,739.00
accuracy,0.87,0.87,0.87,0.87
macro avg,0.88,0.76,0.80,976.00
weighted avg,0.88,0.87,0.86,976.00


In [62]:
from sklearn.metrics.pairwise import cosine_similarity

def classify_text(text, model, label_embeddings):
    text_embedding = generate_embeddings(text, model)

    positive_similarity = cosine_similarity([text_embedding], [label_embeddings[1]])[0][0]
    negative_similarity = cosine_similarity([text_embedding], [label_embeddings[0]])[0][0]

    classification = 'positive' if positive_similarity > negative_similarity else 'negative'
    
    return classification, positive_similarity, negative_similarity

classification_results_list = []

def classify(text):
    global EMBEDDING_MODEL
    global classification_results_list
    global label_embeddings
    
    classification, positive_similarity, negative_similarity = classify_text(
        text,
        EMBEDDING_MODEL,
        label_embeddings
    )
    
    classification_results_list.append({
        'Text': text,
        'Classification': classification,
        'Positive Similarity': "{:.2f}%".format(positive_similarity * 100),
        'Negative Similarity': "{:.2f}%".format(negative_similarity * 100)
    })
    
    classification_df = pd.DataFrame(classification_results_list)
    
    pivot_table = classification_df.pivot_table(
        index='Text',
        values=['Classification', 'Positive Similarity', 'Negative Similarity'],
        aggfunc={'Classification': 'max', 'Positive Similarity': 'first', 'Negative Similarity': 'first'}
    )
    
    pivot_table_sorted = pivot_table.sort_values(by='Positive Similarity', ascending=False)
    
    return pivot_table_sorted

In [63]:
classify("Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın")

,Classification,Negative Similarity,Positive Similarity
Text,,,
Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın,negative,76.42%,73.75%


In [64]:
classify("Başta Hepsiburadaya sonrada mağazaya çok teşekkür ediyorum. Ürünü 1 gün içinde elime ulaştırdılar. Ürünü 1 haftadır kullanıyorum. Çok memnunum. Çok güzel bir ürün. Tavsiye ederim. ")

,Classification,Negative Similarity,Positive Similarity
Text,,,
Başta Hepsiburadaya sonrada mağazaya çok teşekkür ediyorum. Ürünü 1 gün içinde elime ulaştırdılar. Ürünü 1 haftadır kullanıyorum. Çok memnunum. Çok güzel bir ürün. Tavsiye ederim.,positive,73.05%,75.15%
Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın,negative,76.42%,73.75%


In [65]:
classify("Nasıl olurda bu kadar güzel olur. Çok beğendim. Teşekkürler Hepsiburada")

,Classification,Negative Similarity,Positive Similarity
Text,,,
Başta Hepsiburadaya sonrada mağazaya çok teşekkür ediyorum. Ürünü 1 gün içinde elime ulaştırdılar. Ürünü 1 haftadır kullanıyorum. Çok memnunum. Çok güzel bir ürün. Tavsiye ederim.,positive,73.05%,75.15%
Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın,negative,76.42%,73.75%
Nasıl olurda bu kadar güzel olur. Çok beğendim. Teşekkürler Hepsiburada,positive,71.96%,73.68%


In [66]:
classify("cihazı göremedim fakat süreci söyleyeyim berbat hepsiexpressle sakın alışveriş  yapmayın böyle basit bir firma olamaz sabah arayıp sorduğum temsilci kesinlikle mesai bitimine kadar gelecek demesine rağmen gelmedi ve beni bütün gün işe bağladılar bunu beklemek yüzünden böyle rezillik olmaz yazık 3 sefer sordum halbuki mümkünse başka satıcıdan başka firmadan alın")

,Classification,Negative Similarity,Positive Similarity
Text,,,
Başta Hepsiburadaya sonrada mağazaya çok teşekkür ediyorum. Ürünü 1 gün içinde elime ulaştırdılar. Ürünü 1 haftadır kullanıyorum. Çok memnunum. Çok güzel bir ürün. Tavsiye ederim.,positive,73.05%,75.15%
Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın,negative,76.42%,73.75%
Nasıl olurda bu kadar güzel olur. Çok beğendim. Teşekkürler Hepsiburada,positive,71.96%,73.68%
cihazı göremedim fakat süreci söyleyeyim berbat hepsiexpressle sakın alışveriş yapmayın böyle basit bir firma olamaz sabah arayıp sorduğum temsilci kesinlikle mesai bitimine kadar gelecek demesine rağmen gelmedi ve beni bütün gün işe bağladılar bunu beklemek yüzünden böyle rezillik olmaz yazık 3 sefer sordum halbuki mümkünse başka satıcıdan başka firmadan alın,negative,75.48%,73.52%


In [67]:
classify("Geçenlerde eve bir tane daha aldım. Ürünü bizimkiler pek beğenmiş gibi değiller, ne iyi ne de kötü diyebilirim.")

,Classification,Negative Similarity,Positive Similarity
Text,,,
Başta Hepsiburadaya sonrada mağazaya çok teşekkür ediyorum. Ürünü 1 gün içinde elime ulaştırdılar. Ürünü 1 haftadır kullanıyorum. Çok memnunum. Çok güzel bir ürün. Tavsiye ederim.,positive,73.05%,75.15%
Almayın arkadaşlar tavsiye etmiyorum kasıyor ısınıyor çok kötü şarj dayanmıyor almayın,negative,76.42%,73.75%
Nasıl olurda bu kadar güzel olur. Çok beğendim. Teşekkürler Hepsiburada,positive,71.96%,73.68%
cihazı göremedim fakat süreci söyleyeyim berbat hepsiexpressle sakın alışveriş yapmayın böyle basit bir firma olamaz sabah arayıp sorduğum temsilci kesinlikle mesai bitimine kadar gelecek demesine rağmen gelmedi ve beni bütün gün işe bağladılar bunu beklemek yüzünden böyle rezillik olmaz yazık 3 sefer sordum halbuki mümkünse başka satıcıdan başka firmadan alın,negative,75.48%,73.52%
"Geçenlerde eve bir tane daha aldım. Ürünü bizimkiler pek beğenmiş gibi değiller, ne iyi ne de kötü diyebilirim.",positive,73.06%,73.16%
